In [4]:
#prueba de commit en github con el archivo de jupyter lab
filenames = [
    "drivers.csv",
    "timesheet.csv",
    "truck_event_text_partition.csv",
]


In [8]:
#Descargando datos desde repo github
import pandas as pd

url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.csv"
drivers = pd.read_csv(url)
drivers.head()

url2="https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv"
truck_event_text_partition=pd.read(url2)
truck_event_text_partition.head()

,driverId,name,ssn,location,certified,wage-plan
0,10,George Vetticaden,621011971,244-4532 Nulla Rd.,N,miles
1,11,Jamie Engesser,262112338,366-4125 Ac Street,N,miles
2,12,Paul Coddin,198041975,Ap #622-957 Risus. Street,Y,hours
3,13,Joe Niemiec,139907145,2071 Hendrerit. Ave,Y,hours
4,14,Adis Cesir,820812209,Ap #810-1228 In St.,Y,hours


In [11]:
import pandas as pd
url2="https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv"
truck_event_text_partition=pd.read_csv(url2)
truck_event_text_partition.head()

,driverId,truckId,eventTime,eventType,longitude,latitude,eventKey,CorrelationId,driverName,routeId,routeName,eventDate
0,14,25,59:21.4,Normal,-94.58,37.03,14|25|9223370572464814373,3.660000e+18,Adis Cesir,160405074,Joplin to Kansas City Route 2,2016-05-27-22
1,18,16,59:21.7,Normal,-89.66,39.78,18|16|9223370572464814089,3.660000e+18,Grant Liu,1565885487,Springfield to KC Via Hanibal,2016-05-27-22
2,27,105,59:21.7,Normal,-90.21,38.65,27|105|9223370572464814070,3.660000e+18,Mark Lochbihler,1325562373,Springfield to KC Via Columbia Route 2,2016-05-27-22
3,11,74,59:21.7,Normal,-90.20,38.65,11|74|9223370572464814123,3.660000e+18,Jamie Engesser,1567254452,Saint Louis to Memphis Route2,2016-05-27-22
4,22,87,59:21.7,Normal,-90.04,35.19,22|87|9223370572464814101,3.660000e+18,Nadeem Asghar,1198242881,Saint Louis to Chicago Route2,2016-05-27-22
